<div align="center">

---
# Decision Trees - ID3 [Artificial Intelligence Project]
---
</div>

<div align="center">

***
## Problem Presentation
***
</div>
    
> ADD PROBLEM PRESENTATION

<div align="center">

***
## ID3 Algorithm 
***
</div>

A well-known decision tree approach for Machine Learning is the Iterative Dichotomiser 3 (ID3) algorithm. By choosing the best characteristic at each node to partition the data depending on information gain, it recursively constructs a tree. The goal is to make the final subsets as homogeneous as possible. By choosing features that offer the greatest reduction in entropy or uncertainty, ID3 iteratively grows the tree. The procedure keeps going until a halting requirement is satisfied, like a minimum subset size or a maximum tree depth. 

The ID3 Algorithm is specifically designed for building decision trees from a given dataset. It's primary objective is to construct a tree that best explains the relationship between attributes in the data and their corresponding class labels.

**1. Selecting the Best Attribute:**
- ID3 employs the concept of entropy and information gain to determine the attribute that best separates the data. Entropy measures the impurity or randomness in the dataset.
- The algorithm calculates the entropy of each attribute and selects the one that results in the most significant information gain when used for splitting the data.

**2. Creating Tree Nodes:**
- The chosen attribute is used to split the dataset into subsets based on its distinct values.
- For each subset, ID3 recurses to find the next best attribute to further partition the data, forming branches and new nodes accordingly.

**3. Stopping Criteria:**
- The recursion continues until one of the stopping criteria is met, such as when all instances in a branch belong to the same class or when all attributes have been used for splitting.

**4. Handling Missing Values:**
- ID3 can handle missing values to prevent overfitting. While not directly included in ID3, post-processing techniques or variations like C4.5 incorporate pruning to improve the tree's generalization.

<div align="center">

***
## Mathematical Concepts of ID3 Algorithm
***
</div>

### Entropy

**Entropy** is a measure of disorder or uncertainty in a set of data. It is a tool used in ID3 to measure a dataset's disorder  or impurity. By dividing the data into as homogeneous subsets as feasible, the objective is to minimze entropy.

For a set $S$ with classes $\{c_1,\space c_2,\space ...\space,\space c_n \}$, the entropy is calculated as:

$$H(S) = \sum_{i=1}^n \space p_i \space log_2(p_i)$$

Where $p_i$ is the proportion of instances of class $c_i$ in the set.

### Information Gain

Information Gain measures how well a certain quality reduces uncertainty. ID3 splits the data at each stage, choosing the property that maximizes Information Gain. It is computes using the distinction between entropy prior to and following the split.

Information Gain measures the effectiveness of an Attribute $A$ in reducing uncertainty in set $S$

$$IG(A,S) = H(S) - \sum_{v \space \in \space values(A)} \frac{|S_v|}{|S|} \cdot H(S_v))$$

Where, $|S_v|$ is the size of the subset of $S$ for which attribute $A$ has value $v$.

### Gain Ratio

Gain Ratio is an improvement on Information Gain that considers the inherent worth of characteristics that have a wide range of possible values. It deals with the bias of Information Gan in favor of characteristics with more pronounced values.

$$ GR(A,S) = \frac{IG(A,S)}{\sum_{v\space\in\space values(A)} \frac{|S_v|}{|S|} \cdot log_2(\frac{|S_v|}{|S|})} $$

<div align="center">

***
## Problem's Resolution Approach
***
</div>

In [6]:
# Importing Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import (Counter)

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        # Feature and Threshold this node was divided with
        self.feature = feature
        self.threshold = threshold
        
        # Defining the Left and Right children
        self.left = left
        self.right = right

        # Value of a Node -> Determines if it is a Node or not
        self.value = value

    def is_leaf(self):
        # If a Node does not have a Value then it is not a Leaf
        return self.value is not None

In [8]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        # Amount of Samples needed to perform a split
        self.min_samples_split = min_samples_split

        # Max depth of the decision tree
        self.max_depth = max_depth

        # Number of features (X) - Helps add some randomness to the Tree
        self.n_features = n_features

        # Defining a root - will later help to traverse the tree
        self.root = None

    def _most_common_label(self, y):
        # Creating a Counter
        counter = Counter(y)

        # Getting the Most Common Value
        value = counter.most_common(1)[0][0]

        # Returns most common value
        return value

    def _entropy(self, y):
        # The Bincount method creates a numpy array with the occurences of each value.
        # The index of the array is the number and it's value in the array corresponds to the amount of times it appears in y
        occurences = np.bincount(y)

        # Calculating every pi for every X in the previous array
        ps = occurences / len(y)

        # Returning the Entropy Value
        return - np.sum(p * np.log(p) for p in ps if p > 0)

    def _split(self, X_Column, split_threshold):
        # Splitting the Data
        # Note: np.argwhere().flatten() returs the list of indices from the given one where it's elements obey the condition given
        left_indices = np.argwhere(X_Column <= split_threshold).flatten()
        right_indices = np.argwhere(X_Column > split_threshold).flatten()
        return left_indices, right_indices

    def _information_gain(self, y, X_Column, threshold):
        # Getting the Parent Entropy
        parent_entropy = self._entropy(y)

        # Create the Children
        left_indices, right_indices = self._split(X_Column, threshold)

        # Checks if any of the lists are empty
        if (left_indices.size == 0 or right_indices.size == 0):
            return 0

        # -> Calculate the Weighted Average Entropy of the Children

        # Number of Samples in y
        n = len(y)

        # Number of samples in the Left and Right children
        n_left, n_right = left_indices.size, right_indices.size

        # Calculate the Entropy for both Samples (Left and Right)
        entropy_left, entropy_right = self._entropy(y[left_indices]), self._entropy(y[right_indices])

        # Calculate the Child Entropy
        child_entropy = (n_left / n) * entropy_left + (n_right / n) * entropy_right

        # Calculate Information Gain
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _best_split(self, X, y, feature_indices):
        # Finds the Best existent split and threshold (Based on the Information Gain)

        # Initializing the Best Parameters
        best_gain = -1
        split_idx, split_threshold = None, None

        # Traverse all possible actions
        for feat_idx in feature_indices:
            X_Column = X[:, feat_idx]
            thresholds = np.unique(X_Column)

            for threshold in thresholds:
                # Calculate the Information Gain
                gain = self._information_gain(y, X_Column, threshold)

                # Updating the Best Parameters
                if (gain > best_gain):
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = threshold

        # Returning the Best Split Criteria Found
        return split_idx, split_threshold

    def _grow_tree(self, X, y, depth=0):
        # Getting the number of samples, features and labels in the data given
        n_samples, n_features = X.shape
        n_labels = np.unique(y).size

        """
        # Stopping Criteria

        (depth >= self.max_depth)             => Reached Maximum dpeth defined
        (n_labels == 1)                       => Current Node only has 1 type of label (which means it's pure)
        (n_samples < self.min_samples_split)  => The amount of samples is not enough to perform a split

        Therefore, we must return a new node (which is going to be a leaf)
        with the current inform
        """

        # Checks the Stopping Criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Getting the Indices of the Features
        features_indices = np.random.choide(n_features, self.n_features, replace=False)

        # Find the Best Split 
        best_feature, best_threshold = self._best_split(X, y, features_indices)

        # Create Child Nodes (Also makes a recursive call to continue to grow the tree)
        left_indices, right_indices = self._split(X[:, best_feature], best_threshold)
        left = self._grow_tree(X[left_indices, :], y[left_indices], depth + 1)
        right = self._grow_tree(X[right_indices, :], y[right_indices], depth + 1)
        return Node(best_feature, best_threshold, left, right)

        def fit(self, X, y):

            # Making sure that the amount of features does not surpass the ones available
            if not self.n_features:
                self.n_features = X.shape[1]
            else:
                self.n_features = min(X.shape[1], self.n_features)

            # Creating a Tree Recursively
            self.root = self._grow_tree(X, y)

        def _traverse_tree(self, X, node:Node):
            # Traverses the Tree until we reached a leaf node -> which will determine the classification label
            if (node.is_leaf()):
                return node.value

            if (X[node.feature] <= node.threshold):
                return self._traverse_tree(X, node.left)
            else:
                return self._traverse_tree(X, node.right)

        def predict(self, X):
            # Predicts the Label given an Input
            return np.array([self._traverse_tree(x, self.root) for x in X])

        def accuracy(self, y_test, y_predicted):
            # Returns the Accuracy of the Model
            return np.sum(y_test == y_predicted) / len(y_test)

In [9]:
def test_train_split(X, y, test_size=0.3):
    if test_size > 1 or test_size < 0:
        raise Exception("Invalid Test Size Proprotion (Must be between 0 - 1)")

<div align="center">

***
## Bibliographic References
***
</div>

1. Geeks For Geeks (2024). *Iteratice Dichotomiser 3 (ID3) Algorithm From Scratch*. Available [here](https://www.geeksforgeeks.org/iterative-dichotomiser-3-id3-algorithm-from-scratch/)

___
## Final Considerations

$\quad$If there is any difficulty on downloading or executing this project, please contact us via:

- **Email**:
    - [Gonçalo Esteves](https://github.com/EstevesX10) &#8594; `up202203947@up.pt`
    - [Maximino Canhola](https://github.com/MaximinoCanhola) &#8594; `up201909805@up.pt`
    - [Nuno Gomes](https://github.com/NightF0x26) &#8594; `up202206195@up.pt`